# Interact+InspectDR: Bode Plot Example

In [ ]:
using InspectDR

#When MIME"image/svg+xml" is disabled, Jupyter eventually requests PNG inline graphics:
#(SVG outputs do not render well in notebooks for some reason...)
InspectDR.defaults.rendersvg = false

include("BodePlots.jl") #Defines BodePlots.* utilities.

@info("Base utilities loaded.")

# Setup: Interact, Reactive, and Signals
(Also construct widgets and plot object, `pobj`)

## TODO

 - Find out why `txt_fmax` `spinbox` cannot be constructed without making Julia hang..
 - Improve widget layout (`hstack`?) once supported by Interact.

In [ ]:
using Interact

#Create base Observable-s that control plot output:
fmult = Observable(Float64(1e9)) #For all user-input
fmax = Observable(Float64(10)) #Maximum plot frequency
G = Observable(Float64(20)) #Gain (dB)
fz = Observable(Float64(fmax[])) #Zero frequency
fp1 = Observable(Float64(.05)) #Pole 1 frequency
fp2 = Observable(Float64(.9)) #Pole 2 frequency
annot = Observable(Bool(true)) #Annotate plot

#Create initial plot object:
pobj = BodePlots.new()

#Define desired frequency ranges:
fmult_opt = OrderedDict("kHz" => 1.0e3, "MHz" => 1.0e6, "GHz" => 1.0e9) #Make sure to use Float64

function frequency_slider(sig::Observable, fmax::Observable; label::String="")
    _fmin = 1e-3
    rng(_fmax) = 10 .^ range(log10(_fmin), stop=log10(max(_fmin, _fmax)), length=100)
    #NOTE: Limit max value to 2_fmin to avoid issues:
    return map((_fmax)->slider(rng(_fmax), value=sig, label=label), fmax)
end

#Construct widgets used to control plots:
tgl_fmult = togglebuttons(fmult_opt, value=fmult, label="freq multiplier")
#txt_fmax = spinbox(value=fmax, label="max freq") #Causes Julia to hang.
txt_fmax = fmax #Workaround
sld_G = slider(-10:1.0:100, value=G, label="Gain (dB)")
sld_fz = frequency_slider(fz, fmax, label="zero")
sld_fp1 = frequency_slider(fp1, fmax, label="pole 1")
sld_fp2 = frequency_slider(fp2, fmax, label="pole 2")
chk_annot = checkbox(annot, label="Display annotation")

#Build widget list to control plot for future code blocks
widgetlist = [tgl_fmult, txt_fmax, sld_G, sld_fz, sld_fp1, sld_fp2, chk_annot]
#widgetlist = [txt_fmax, sld_fz, sld_fp1, sld_fp2]

@info("Widgets constructed.")

# Interact/Reactive Control: Inline Plots
Use sliders/text boxes to alter transfer function

In [ ]:
#Widget-controlled plot:
updater_plot = map(fmult, fmax, G, fz, fp1, fp2, annot) do s, _fmax, _G, _fz, _fp1, _fp2, a
    BodePlots.update(pobj, s*1e-3, _fmax*s, _G, _fz*s, _fp1*s, _fp2*s, a)
end
display(updater_plot)
map(display, widgetlist)

@info("Ready to control inline plot.")

# Interact/Reactive Control: Inspect/Gtk GUI
(Re-display GUI controls for convenience)

In [ ]:
gtkgui = display(InspectDR.GtkDisplay(), pobj) #Display plot in Gtk GUI.

#Refresh GUI when plot changes:
updater_gtkgui = map((p)->InspectDR.refresh(gtkgui), updater_plot)

#Re-display GUI controls, for convenience:
map(display, widgetlist)

@info("Ready to control Gtk plot.")

# Demo complete!